In [1]:
import torch
import augdataset
import loss, train_utils, models
import utils

In [2]:
import logging
logging.basicConfig(filename='train_status.log',
                            filemode='a',
                            format='%(asctime)s %(message)s',
                            datefmt='%H:%M:%S',
                            level=logging.INFO)

In [3]:
# get the trainig data
train_dataloader = augdataset.get_train_dl(dirs=['/home/devi_prasad/UrineSedimentation/data_imgs/10k_annotatio_details_20200901/confusing/*.*'])

Number of Images:  4356
Compose(
    Resize(size=(150, 150), interpolation=PIL.Image.BILINEAR)
    RandomResizedCrop(size=(150, 150), scale=(0.2, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
    RandomHorizontalFlip(p=0.5)
    RandomVerticalFlip(p=0.5)
    RandomApply(
    p=0.8
    ColorJitter(brightness=[0.6, 1.4], contrast=[0.6, 1.4], saturation=[0.6, 1.4], hue=[-0.1, 0.1])
)
    RandomGrayscale(p=0.2)
    ToTensor()
    Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.201])
)


In [4]:
# import model 
# output feature size = 128
model = models.Model(features_dim=128)
model = model.cuda()
# resume training
# model = torch.load('models/modelv1-Copy1.0.pt')

In [5]:
# for param in model.parameters():
#     print(param.requires_grad)# = True

In [6]:
v = 2
# SimCLR loss https://arxiv.org/pdf/2002.05709.pdf
criterion = loss.SimCLRLoss(temperature=0.1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# decay the learning rate by 0.1 after every 500 epochs
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 200, gamma=0.1, last_epoch=-1,)

# decay by .1 if no improvement for 100 epochs
scheduler = utils.ReduceLROnPlateauWithBacktrack(optimizer, model, filename=f'models/modelv{v}.pt', factor=0.1, verbose=False, patience=100, warmup_steps=0, eps=1e-8)
n_epochs = 5000

In [ ]:
# train model
for epoch in range(n_epochs):
    
    total_loss = train_utils.train(train_dataloader, model, criterion, optimizer)
    
    print(f"Epoch {epoch} total_loss {total_loss} lr : {optimizer.param_groups[0]['lr']}", )
    
    logging.info(f"modelv{v} Epoch {epoch} total_loss {total_loss} lr {optimizer.param_groups[0]['lr']}")
    scheduler.step([-total_loss])
    
    torch.save([model, optimizer], f'models/modelv{v}_curr.pt')

Epoch 0 total_loss 67.42889088392258 lr : 0.0016
Epoch 1 total_loss 44.26941004395485 lr : 0.00175
Epoch 2 total_loss 35.78485497832298 lr : 0.001
Epoch 3 total_loss 31.17360121011734 lr : 0.001
Epoch 4 total_loss 28.72191560268402 lr : 0.001
Epoch 5 total_loss 27.110676765441895 lr : 0.001
Epoch 6 total_loss 25.084012486040592 lr : 0.001
Epoch 7 total_loss 24.755797296762466 lr : 0.001
Epoch 8 total_loss 24.886099964380264 lr : 0.001
Epoch 9 total_loss 23.581283420324326 lr : 0.001
Epoch 10 total_loss 22.61112755537033 lr : 0.0013
Epoch 11 total_loss 21.113924026489258 lr : 0.001
Epoch 12 total_loss 21.580912321805954 lr : 0.001
Epoch 13 total_loss 20.613198846578598 lr : 0.001
Epoch 14 total_loss 20.189198911190033 lr : 0.001
Epoch 15 total_loss 20.252035975456238 lr : 0.001
Epoch 16 total_loss 19.524524301290512 lr : 0.001
Epoch 17 total_loss 18.586153157055378 lr : 0.0016
Epoch 18 total_loss 18.18667009472847 lr : 0.001
Epoch 19 total_loss 17.95823523402214 lr : 0.0012
Epoch 20 tot